In [1]:
import pandas as pd
from training_data import import_training_set

data, weights, rulesetIds = import_training_set("components classification")

In [2]:
data

,Selection 1 - ProgressiveHistory,Selection 1 - Random,Selection 1 - UCB1,Selection 1 - UCB1GRAVE,Selection 1 - UCB1Tuned,Selection 2 - ProgressiveHistory,Selection 2 - Random,Selection 2 - UCB1,Selection 2 - UCB1GRAVE,Selection 2 - UCB1Tuned,...,CrossBoard,KintsBoard,PachisiBoard,FortyStonesWithFourGapsBoard,DiceD2,DiceD4,DiceD6,PlayersWithDirections,IsPieceAt,Most common outcome for Agent 1
0,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,Win
1,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,Win
2,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,Loss
3,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,Win
4,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,Loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17877,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,Win
17878,False,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,Win
17879,False,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,Win
17880,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,True,False,Loss


In [3]:
weights

array([0.61284247, 0.61284247, 0.61284247, ..., 0.9875    , 0.9875    ,
       0.9875    ])

In [4]:
rulesetIds

array(['37', '37', '37', ..., '456', '456', '456'], dtype='<U21')

In [5]:
import time
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# Evaluate the model ruleset by ruleset -> assume that the model has not seen the supplied ruleset before

# Prepare data
X = data.drop(columns=["Most common outcome for Agent 1"])
y = data["Most common outcome for Agent 1"]

# Custom implementation of Leave-One-Out Cross-Validation
y_true = []
y_pred = []

startTime = time.time()
noIterations = 0
noRulesets = np.unique(rulesetIds).shape[0]

for rulesetId in np.unique(rulesetIds):
    # Train/test split
    X_train = X[rulesetIds != rulesetId]
    y_train = y[rulesetIds != rulesetId]

    X_test = X[rulesetIds == rulesetId]
    y_test = y[rulesetIds == rulesetId]

    # Train classifier
    clf = DecisionTreeClassifier().fit(X_train, y_train)

    # Predict on the test instance
    y_true += list(y_test)
    y_pred += list(clf.predict(X_test))

    # Output progress
    noIterations += 1
    if noIterations % 10 == 0:
        elapsedTime = time.time() - startTime
        avgTimePerIteration = elapsedTime / noIterations
        print(f"Progress: {noIterations}/{noRulesets}; Time elapsed (s): {elapsedTime:.2f}; Estimated time left (s): {(noRulesets - noIterations) * avgTimePerIteration:.2f}")

# for i in range(X.shape[0]):
#     # Train/test split
#     X_train = X.drop(index=[i])
#     y_train = y.drop(index=[i])
#     
#     X_test = X.loc[i]
#     
#     # Train classifier
#     clf = DecisionTreeClassifier().fit(X_train, y_train)
#     
#     # Predict on the test instance
#     y_pred.append(clf.predict(pd.DataFrame(X_test).transpose()))
#     
#     # Output progress
#     if (i + 1) % 1000 == 0:
#         elapsedTime = time.time() - startTime
#         avgTimePerInstance = elapsedTime / (i + 1)
#         print(f"Progress: {i + 1}/{X.shape[0]}; Time elapsed (s): {elapsedTime:.2f}; Estimated time left (s): {(X.shape[0] - (i + 1)) * avgTimePerInstance:.2f}")

print("Progress: done")

print(f"F1-score: {f1_score(y_true=y_true, y_pred=y_pred)}")

Progress: 10/1017; Time elapsed (s): 4.73; Estimated time left (s): 475.97
Progress: 20/1017; Time elapsed (s): 9.15; Estimated time left (s): 455.94
Progress: 30/1017; Time elapsed (s): 13.19; Estimated time left (s): 433.88
Progress: 40/1017; Time elapsed (s): 17.20; Estimated time left (s): 420.04
Progress: 50/1017; Time elapsed (s): 21.17; Estimated time left (s): 409.48
Progress: 60/1017; Time elapsed (s): 25.19; Estimated time left (s): 401.82
Progress: 70/1017; Time elapsed (s): 29.41; Estimated time left (s): 397.92
Progress: 80/1017; Time elapsed (s): 33.53; Estimated time left (s): 392.72
Progress: 90/1017; Time elapsed (s): 37.89; Estimated time left (s): 390.29
Progress: 100/1017; Time elapsed (s): 42.28; Estimated time left (s): 387.66
Progress: 110/1017; Time elapsed (s): 46.38; Estimated time left (s): 382.43
Progress: 120/1017; Time elapsed (s): 50.45; Estimated time left (s): 377.13
Progress: 130/1017; Time elapsed (s): 54.60; Estimated time left (s): 372.52
Progress: 

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [7]:
print(f"F1-score: {f1_score(y_true=y_true, y_pred=y_pred, average='weighted')}")

F1-score: 0.6763010040707458
